<a href="https://colab.research.google.com/github/NaamaSchweitzer/CV-waste-classification/blob/Preprocessing/CV_Waste_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Kaggle Setup and Dataset Download

This section will guide you through setting up Kaggle in your Colab environment and downloading the `alyyan/trash-detection` dataset.

In [1]:
import os

# Install the Kaggle API client
!pip install kaggle


#### Authenticate with Kaggle

To download datasets from Kaggle, you need to authenticate using your Kaggle API token. Follow these steps:

1.  Go to your Kaggle account page (https://www.kaggle.com/your-username/account).
2.  Scroll down to the 'API' section and click 'Create New API Token'. This will download a `kaggle.json` file.
3.  In Colab, click on the 🔑 icon (Secrets) in the left sidebar. Add a new secret named `KAGGLE_USERNAME` for your Kaggle username and `KAGGLE_KEY` for your Kaggle API key (from the `kaggle.json` file).

Alternatively, you can upload the `kaggle.json` file directly:


In [2]:
# Option 1: Using Colab Secrets (Recommended)
from google.colab import userdata
import os

# Ensure these secrets are set in Colab's Secrets manager
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

# Option 2: Uploading kaggle.json directly (uncomment and run if not using secrets)
# from google.colab import files
# files.upload() # This will prompt you to upload the kaggle.json file

# Create .kaggle directory if it doesn't exist and move the file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json # Set permissions

print("Kaggle authentication setup complete.")

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Kaggle authentication setup complete.


#### Download the Dataset

Now we will download the `trash-detection` dataset from Kaggle.

In [3]:
# Download the dataset
!kaggle datasets download -d alyyan/trash-detection

# List the downloaded file(s) to verify
!ls


Dataset URL: https://www.kaggle.com/datasets/alyyan/trash-detection
License(s): MIT
100% 1.23G/1.23G [00:11<00:00, 211MB/s]
100% 1.23G/1.23G [00:11<00:00, 111MB/s]
sample_data  trash-detection.zip


#### Unzip the Dataset

The dataset is typically downloaded as a zip file. Let's extract its contents.

In [4]:
# Unzip the dataset (assuming it's a .zip file)
!unzip -q trash-detection.zip -d trash-detection

# List the contents of the extracted directory
!ls trash-detection

print("Dataset downloaded and extracted successfully!")

Dataset
Dataset downloaded and extracted successfully!


# Task
Analyze the structure of the `trash-detection/Dataset` directory, read the `trash-detection/Dataset/data.yaml` file to extract class names, count the total instances and analyze the class balance by processing the label files within the dataset, and finally provide a comprehensive summary of the dataset including its overall size, detected classes, and the distribution of instances across these classes.

## Explore Dataset Structure

### Subtask:
List the contents of the `trash-detection/Dataset` directory to understand how the images and labels are organized.


**Reasoning**:
To understand how the images and labels are organized, I will list the contents of the `trash-detection/Dataset` directory using a shell command.



In [5]:
import os

# List the contents of the 'trash-detection/Dataset' directory
!ls -F trash-detection/Dataset

data.yaml  images/  labels/


## Read Data Configuration

### Subtask:
Load and parse the `trash-detection/Dataset/data.yaml` file to extract class names and any other relevant dataset metadata.


**Reasoning**:
Before parsing the YAML file, the `pyyaml` library needs to be installed, as it's not part of the standard Python library and was not installed in previous steps.



In [6]:
import os

# Install the PyYAML library to parse YAML files
!pip install pyyaml

print("PyYAML installed successfully.")

PyYAML installed successfully.


**Reasoning**:
Now that PyYAML is installed, I will import the `yaml` library, construct the path to `data.yaml`, open and parse it using `yaml.safe_load`, and then extract the class names as instructed.



In [7]:
import yaml

# Construct the full path to the data.yaml file
data_yaml_path = 'trash-detection/Dataset/data.yaml'

# Initialize variables
class_names = []

# Open and read the data.yaml file
with open(data_yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Extract class names
if 'names' in data:
    class_names = data['names']
elif 'nc' in data and 'names' in data:
    class_names = data['names']

print(f"Dataset metadata loaded from: {data_yaml_path}")
print(f"Detected classes (nc): {len(class_names)}")
print(f"Class names: {class_names}")

Dataset metadata loaded from: trash-detection/Dataset/data.yaml
Detected classes (nc): 4
Class names: ['dirt', 'liquid', 'marks', 'trash']


## Count Total Instances and Analyze Class Balance

### Subtask:
Identify the label files (e.g., .txt files) within the dataset and count the total number of labels. Then, iterate through these label files to count the occurrences of each class, providing a breakdown of the class balance.


**Reasoning**:
I will implement the instructions to identify label files, count total instances, and analyze class balance by iterating through the label directory, reading each label file, extracting class IDs, and then aggregating the counts for each class.



# Task
Count the total number of labels in `trash-detection/Dataset/labels`, and then parse the class ID from each line in these label files (which are in YOLO annotation format) to count the occurrences of each class and analyze the class balance.

## Count Total Instances and Analyze Class Balance

### Subtask:
Identify the label files (e.g., .txt files) within the dataset, which are in YOLO annotation format. Iterate through these label files, parse the class ID from each line (the first element), and count the total number of instances and the occurrences of each class to provide a breakdown of the class balance.


**Reasoning**:
To count total instances and analyze class balance, I will define the labels directory, initialize counters, iterate through all .txt label files, extract class IDs from each line, and aggregate the counts.



In [8]:
import os

# Define the path to the labels directory
labels_dir = './trash-detection/Dataset/labels/train'

# Initialize an empty dictionary to store class counts
# and a variable to store the total number of instances
class_counts = {i: 0 for i in range(len(class_names))}
total_instances = 0

# List all files in the labels directory
all_label_files = os.listdir(labels_dir)

# Filter for .txt files (YOLO label files)
label_files = [f for f in all_label_files if f.endswith('.txt')]

print(f"Processing {len(label_files)} label files...")

# Iterate through each identified label file
for filename in label_files:
    filepath = os.path.join(labels_dir, filename)
    with open(filepath, 'r') as file:
        for line in file:
            # Increment total instances for each line (each bounding box)
            total_instances += 1

            # Extract class ID (first element in the line)
            try:
                class_id = int(line.strip().split(' ')[0])
                # Increment the count for this class ID
                class_counts[class_id] = class_counts.get(class_id, 0) + 1
            except ValueError as e:
                print(f"Warning: Could not parse class ID in file {filename}, line: {line.strip()}. Error: {e}")
            except IndexError as e:
                print(f"Warning: Empty line or unexpected format in file {filename}, line: {line.strip()}. Error: {e}")

print("\n--- Dataset Class Analysis ---")
print(f"Total instances (bounding boxes detected): {total_instances}")
print("Class distribution:")

# Print class names with their counts
for class_id, count in class_counts.items():
    if class_id < len(class_names):
        class_name = class_names[class_id]
        print(f"  - {class_name} (ID {class_id}): {count} instances")
    else:
        print(f"  - Unknown Class (ID {class_id}): {count} instances (not in data.yaml)")

Processing 1229 label files...

--- Dataset Class Analysis ---
Total instances (bounding boxes detected): 7900
Class distribution:
  - dirt (ID 0): 1418 instances
  - liquid (ID 1): 494 instances
  - marks (ID 2): 3373 instances
  - trash (ID 3): 2615 instances


### Dataset Summary

This section provides a summary of the `trash-detection` dataset, based on the analysis of its structure and label files.

-   **Overall Dataset Size:**
    -   Total number of label files processed: **1229**
    -   Total number of instances (bounding boxes) detected: **7900**

-   **Detected Classes:**
    -   The dataset contains **4** distinct classes.
    -   Class Names: `['dirt', 'liquid', 'marks', 'trash']`

-   **Class Distribution (Instance Count per Class):**
    -   `dirt`: 1418 instances
    -   `liquid`: 494 instances
    -   `marks`: 3373 instances
    -   `trash`: 2615 instances

This summary indicates that the `marks` class has the highest number of instances, followed by `trash`, `dirt`, and `liquid` having the least.

In [9]:
import os

# Define the path to the labels directory
labels_dir = './trash-detection/Dataset/labels/val'

# Initialize an empty dictionary to store class counts
# and a variable to store the total number of instances
class_counts = {i: 0 for i in range(len(class_names))}
total_instances = 0

# List all files in the labels directory
all_label_files = os.listdir(labels_dir)

# Filter for .txt files (YOLO label files)
label_files = [f for f in all_label_files if f.endswith('.txt')]

print(f"Processing {len(label_files)} label files...")

# Iterate through each identified label file
for filename in label_files:
    filepath = os.path.join(labels_dir, filename)
    with open(filepath, 'r') as file:
        for line in file:
            # Increment total instances for each line (each bounding box)
            total_instances += 1

            # Extract class ID (first element in the line)
            try:
                class_id = int(line.strip().split(' ')[0])
                # Increment the count for this class ID
                class_counts[class_id] = class_counts.get(class_id, 0) + 1
            except ValueError as e:
                print(f"Warning: Could not parse class ID in file {filename}, line: {line.strip()}. Error: {e}")
            except IndexError as e:
                print(f"Warning: Empty line or unexpected format in file {filename}, line: {line.strip()}. Error: {e}")

print("\n--- Dataset Class Analysis ---")
print(f"Total instances (bounding boxes detected): {total_instances}")
print("Class distribution:")

# Print class names with their counts
for class_id, count in class_counts.items():
    if class_id < len(class_names):
        class_name = class_names[class_id]
        print(f"  - {class_name} (ID {class_id}): {count} instances")
    else:
        print(f"  - Unknown Class (ID {class_id}): {count} instances (not in data.yaml)")

Processing 308 label files...

--- Dataset Class Analysis ---
Total instances (bounding boxes detected): 2018
Class distribution:
  - dirt (ID 0): 367 instances
  - liquid (ID 1): 140 instances
  - marks (ID 2): 822 instances
  - trash (ID 3): 689 instances


# Task
Identify the minority classes from the training set based on their instance counts, aiming to balance the dataset by targeting classes with significantly fewer instances for augmentation. Specifically, calculate the average instance count across all classes in the training set and identify classes that fall below this average as candidates for augmentation, outputting the names of these classes.

## Identify Minority Classes for Augmentation

### Subtask:
Using the previously determined class counts for the training set, identify which classes are underrepresented and require data augmentation to achieve a more balanced distribution. This step will define the target classes for augmentation.


**Reasoning**:
I need to calculate the average instance count for the training set and then identify classes with counts below this average to determine minority classes for augmentation, as per the instructions.



In [10]:
import numpy as np

# Class counts for the training set (from previous analysis in cell 1bc2ff07)
train_class_counts = {0: 1418, 1: 494, 2: 3373, 3: 2615}

# Total number of instances in the training set
total_train_instances = sum(train_class_counts.values())

# Number of classes
num_classes = len(class_names)

# Calculate the average instance count per class
average_instance_count = total_train_instances / num_classes

# Initialize a list to store minority class names
minority_classes = []

# Identify minority classes
for class_id, count in train_class_counts.items():
    if count < average_instance_count:
        minority_classes.append(class_names[class_id])

print(f"Total instances in training set: {total_train_instances}")
print(f"Number of classes: {num_classes}")
print(f"Average instance count per class: {average_instance_count:.2f}")
print(f"Identified minority classes for augmentation: {minority_classes}")

Total instances in training set: 7900
Number of classes: 4
Average instance count per class: 1975.00
Identified minority classes for augmentation: ['dirt', 'liquid']


## Prepare Output Directories for Augmented Data

### Subtask:
Create new directories for storing the augmented images and their corresponding label files. This ensures that the original dataset remains untouched and allows for clear separation of augmented data.


**Reasoning**:
I need to define the paths for the original and augmented training image and label directories and then create the augmented directories using `os.makedirs`.



In [11]:
import os

# Define base paths for original training data
original_train_images_dir = './trash-detection/Dataset/images/train'
original_train_labels_dir = './trash-detection/Dataset/labels/train'

# Define new paths for augmented training data
augmented_train_images_dir = './trash-detection/Dataset/images/augmented_train'
augmented_train_labels_dir = './trash-detection/Dataset/labels/augmented_train'

# Create directories for augmented images and labels
os.makedirs(augmented_train_images_dir, exist_ok=True)
os.makedirs(augmented_train_labels_dir, exist_ok=True)

print(f"Created directory: {augmented_train_images_dir}")
print(f"Created directory: {augmented_train_labels_dir}")

Created directory: ./trash-detection/Dataset/images/augmented_train
Created directory: ./trash-detection/Dataset/labels/augmented_train


## Implement Data Augmentation for Images and YOLO Labels

### Subtask:
Develop Python code to perform data augmentation specifically for the identified minority classes. This will involve loading images and their associated YOLO format label files, applying geometric transformations (horizontal/vertical flips, rotations) and photometric transformations (brightness changes), crucially transforming the bounding box coordinates within the YOLO label files to match the applied image augmentations, and saving the augmented images and their transformed label files into the newly created output directories.


**Reasoning**:
I will start by importing the necessary libraries for image manipulation (PIL), numerical operations (numpy), file system interactions (os), and randomness (random). These libraries are fundamental for developing the data augmentation functions and handling file paths.



In [12]:
import os
import numpy as np
from PIL import Image, ImageEnhance
import random

print("Necessary libraries imported: os, numpy, PIL (Image, ImageEnhance), random.")

Necessary libraries imported: os, numpy, PIL (Image, ImageEnhance), random.


**Reasoning**:
I will define utility functions to read and write YOLO label files, as these are crucial for parsing existing bounding box annotations and saving transformed ones after augmentation.



In [13]:
def read_yolo_labels(label_filepath):
    """Reads a YOLO format label file and returns a list of bounding boxes.
    Each bounding box is [class_id, x_center, y_center, width, height] (normalized).
    """
    bboxes = []
    if not os.path.exists(label_filepath):
        return bboxes
    with open(label_filepath, 'r') as f:
        for line in f:
            parts = list(map(float, line.strip().split()))
            if len(parts) == 5:
                bboxes.append(parts)
    return bboxes

def write_yolo_labels(label_filepath, bboxes):
    """Writes a list of bounding boxes to a YOLO format label file.
    Each bounding box is [class_id, x_center, y_center, width, height] (normalized).
    """
    with open(label_filepath, 'w') as f:
        for bbox in bboxes:
            # Ensure class_id is an integer, others are floats formatted to 6 decimal places
            f.write(f"{int(bbox[0])} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f} {bbox[4]:.6f}\n")

print("Utility functions 'read_yolo_labels' and 'write_yolo_labels' defined.")

Utility functions 'read_yolo_labels' and 'write_yolo_labels' defined.


**Reasoning**:
I will define the geometric augmentation functions (`horizontal_flip`, `vertical_flip`, `rotate_90_clockwise`) that transform both the image and its corresponding YOLO bounding box coordinates, and a photometric augmentation function (`adjust_brightness`) that only modifies the image, as instructed. These functions are crucial for preparing the data for augmentation.



In [14]:
def horizontal_flip(image, bboxes):
    """Flips the image horizontally and updates bounding box coordinates."""
    # Flip image
    flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # Update bounding box coordinates
    flipped_bboxes = []
    for bbox in bboxes:
        class_id, x_center, y_center, width, height = bbox
        new_x_center = 1 - x_center  # Flip x-coordinate
        flipped_bboxes.append([class_id, new_x_center, y_center, width, height])
    return flipped_image, flipped_bboxes

def vertical_flip(image, bboxes):
    """Flips the image vertically and updates bounding box coordinates."""
    # Flip image
    flipped_image = image.transpose(Image.FLIP_TOP_BOTTOM)

    # Update bounding box coordinates
    flipped_bboxes = []
    for bbox in bboxes:
        class_id, x_center, y_center, width, height = bbox
        new_y_center = 1 - y_center  # Flip y-coordinate
        flipped_bboxes.append([class_id, x_center, new_y_center, width, height])
    return flipped_image, flipped_bboxes

def rotate_90_clockwise(image, bboxes):
    """Rotates the image 90 degrees clockwise and updates bounding box coordinates."""
    # Rotate image
    rotated_image = image.transpose(Image.ROTATE_270) # ROTATE_270 is 90 degrees clockwise

    # Update bounding box coordinates
    rotated_bboxes = []
    for bbox in bboxes:
        class_id, x_center, y_center, width, height = bbox
        # New x_center becomes 1 - original y_center
        # New y_center becomes original x_center
        # New width becomes original height
        # New height becomes original width
        new_x_center = 1 - y_center
        new_y_center = x_center
        new_width = height
        new_height = width
        rotated_bboxes.append([class_id, new_x_center, new_y_center, new_width, new_height])
    return rotated_image, rotated_bboxes

def adjust_brightness(image, bboxes, factor):
    """Adjusts image brightness. Bounding boxes remain unchanged."""
    enhancer = ImageEnhance.Brightness(image)
    brightened_image = enhancer.enhance(factor)
    return brightened_image, bboxes

print("Data augmentation functions (horizontal_flip, vertical_flip, rotate_90_clockwise, adjust_brightness) defined.")

Data augmentation functions (horizontal_flip, vertical_flip, rotate_90_clockwise, adjust_brightness) defined.


**Reasoning**:
I will now iterate through the original training images and their labels, identify images containing minority classes, apply the defined augmentation techniques (horizontal flip, 90-degree clockwise rotation, and brightness adjustment) to these images and their bounding boxes, and save the augmented images and their corresponding transformed label files to the designated augmented directories.



In [15]:
import os
import numpy as np
from PIL import Image, ImageEnhance
import random

# Assuming class_names, minority_classes, original_train_images_dir, original_train_labels_dir,
# augmented_train_images_dir, augmented_train_labels_dir are defined from previous steps.

print(f"Starting data augmentation for minority classes: {minority_classes}")
print(f"Original images directory: {original_train_images_dir}")
print(f"Original labels directory: {original_train_labels_dir}")
print(f"Augmented images directory: {augmented_train_images_dir}")
print(f"Augmented labels directory: {augmented_train_labels_dir}")

augmented_count = 0

# Get a list of all image files in the original training directory
image_files = [f for f in os.listdir(original_train_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for img_filename in image_files:
    base_filename, _ = os.path.splitext(img_filename)
    img_filepath = os.path.join(original_train_images_dir, img_filename)
    label_filepath = os.path.join(original_train_labels_dir, base_filename + '.txt')

    # Load image and labels
    try:
        image = Image.open(img_filepath).convert('RGB')
        bboxes = read_yolo_labels(label_filepath)
    except Exception as e:
        print(f"Error loading {img_filepath} or {label_filepath}: {e}")
        continue

    if not bboxes:
        # Skip if no bounding boxes are found
        continue

    # Check if this image contains any minority class instances
    contains_minority_class = False
    for bbox in bboxes:
        class_id = int(bbox[0])
        if class_names[class_id] in minority_classes:
            contains_minority_class = True
            break

    # Only augment images that contain minority classes
    if contains_minority_class:
        # --- Apply Augmentations ---

        # 1. Horizontal Flip
        flipped_image, flipped_bboxes = horizontal_flip(image, bboxes)
        flipped_img_filename = f"{base_filename}_hf.jpg"
        flipped_label_filename = f"{base_filename}_hf.txt"
        flipped_image.save(os.path.join(augmented_train_images_dir, flipped_img_filename))
        write_yolo_labels(os.path.join(augmented_train_labels_dir, flipped_label_filename), flipped_bboxes)
        augmented_count += 1

        # 2. Rotate 90 degrees clockwise
        rotated_image, rotated_bboxes = rotate_90_clockwise(image, bboxes)
        rotated_img_filename = f"{base_filename}_r90.jpg"
        rotated_label_filename = f"{base_filename}_r90.txt"
        rotated_image.save(os.path.join(augmented_train_images_dir, rotated_img_filename))
        write_yolo_labels(os.path.join(augmented_train_labels_dir, rotated_label_filename), rotated_bboxes)
        augmented_count += 1

        # 3. Adjust Brightness (randomly dim or brighten)
        brightness_factor = random.choice([0.4,0.5,0.6,0.7,0.8,1.2,1.3,1.4,1.5,1.6]) # Factor between 0.7 (dim) and 1.3 (bright)
        brightened_image, brightened_bboxes = adjust_brightness(image, bboxes, brightness_factor)
        brightened_img_filename = f"{base_filename}_br.jpg"
        brightened_label_filename = f"{base_filename}_br.txt"
        brightened_image.save(os.path.join(augmented_train_images_dir, brightened_img_filename))
        write_yolo_labels(os.path.join(augmented_train_labels_dir, brightened_label_filename), brightened_bboxes)
        augmented_count += 1

print(f"Finished data augmentation. Total augmented images/labels generated: {augmented_count}")
print(f"Augmented data stored in: {augmented_train_images_dir} and {augmented_train_labels_dir}")

Starting data augmentation for minority classes: ['dirt', 'liquid']
Original images directory: ./trash-detection/Dataset/images/train
Original labels directory: ./trash-detection/Dataset/labels/train
Augmented images directory: ./trash-detection/Dataset/images/augmented_train
Augmented labels directory: ./trash-detection/Dataset/labels/augmented_train
Finished data augmentation. Total augmented images/labels generated: 1653
Augmented data stored in: ./trash-detection/Dataset/images/augmented_train and ./trash-detection/Dataset/labels/augmented_train


## Verify New Class Balance Post-Augmentation

### Subtask:
After applying augmentations, re-run the class counting logic on the combined original and augmented datasets. This step will confirm whether the dataset has achieved a more balanced distribution across classes, providing updated instance counts for each class.


**Reasoning**:
I will re-run the class counting logic to include the newly augmented data, aggregating counts from both the original and augmented datasets to verify the new class balance, and then print a summary of the combined distribution.



In [16]:
import os

# 1. Initialize combined_class_counts with original training counts
# train_class_counts is already defined from previous steps (cell abd43685)
combined_class_counts = train_class_counts.copy()

# Initialize a variable to store the total number of instances from augmented data
total_augmented_instances = 0

# 2. Define the path to the augmented labels directory
# augmented_train_labels_dir is already defined from previous steps (cell 867f34f8)

print(f"Processing augmented labels from: {augmented_train_labels_dir}")

# 3. Get a list of all .txt files (label files) within the augmented_train_labels_dir
all_augmented_label_files = os.listdir(augmented_train_labels_dir)
augmented_label_files = [f for f in all_augmented_label_files if f.endswith('.txt')]

print(f"Found {len(augmented_label_files)} augmented label files.")

# 4. Iterate through each augmented label file
for filename in augmented_label_files:
    filepath = os.path.join(augmented_train_labels_dir, filename)
    with open(filepath, 'r') as file:
        for line in file:
            # Increment total augmented instances for each line (each bounding box)
            total_augmented_instances += 1

            # Extract class ID (first element in the line)
            try:
                class_id = int(line.strip().split(' ')[0])
                # Increment the count for this class ID in the combined dictionary
                combined_class_counts[class_id] = combined_class_counts.get(class_id, 0) + 1
            except ValueError as e:
                print(f"Warning: Could not parse class ID in augmented file {filename}, line: {line.strip()}. Error: {e}")
            except IndexError as e:
                print(f"Warning: Empty line or unexpected format in augmented file {filename}, line: {line.strip()}. Error: {e}")

# 5. Print the total number of instances from the augmented data
print(f"\nTotal instances added from augmented data: {total_augmented_instances}")

# Calculate total instances in the combined dataset
total_combined_instances = sum(combined_class_counts.values())

print("\n--- Combined Dataset Class Analysis (Original + Augmented) ---")
print(f"Total instances (bounding boxes detected) in combined dataset: {total_combined_instances}")
print("Combined Class distribution:")

# 6. Print the combined_class_counts dictionary
for class_id, count in combined_class_counts.items():
    if class_id < len(class_names):
        class_name_str = class_names[class_id]
        print(f"  - {class_name_str} (ID {class_id}): {count} instances")
    else:
        print(f"  - Unknown Class (ID {class_id}): {count} instances (not in data.yaml)")

# 7. Print a summary indicating the improvement in class balance
print("\n--- Class Balance Improvement Summary ---")
print("Original training class distribution:")
for class_id, count in train_class_counts.items():
    print(f"  - {class_names[class_id]} (ID {class_id}): {count} instances")


# Calculate average instance count for the combined dataset for comparison
average_combined_instance_count = total_combined_instances / num_classes

print(f"\nAverage instance count per class in original training set: {average_instance_count:.2f}")
print(f"Average instance count per class in combined dataset: {average_combined_instance_count:.2f}")

new_minority_classes = []
for class_id, count in combined_class_counts.items():
    if count < average_combined_instance_count:
        new_minority_classes.append(class_names[class_id])

if not new_minority_classes:
    print("All classes are now above or very close to the average instance count, indicating improved balance.")
else:
    print(f"Classes still below the new average: {new_minority_classes}")
    print("Further augmentation might be beneficial for these classes.")


Processing augmented labels from: ./trash-detection/Dataset/labels/augmented_train
Found 1653 augmented label files.

Total instances added from augmented data: 14193

--- Combined Dataset Class Analysis (Original + Augmented) ---
Total instances (bounding boxes detected) in combined dataset: 22093
Combined Class distribution:
  - dirt (ID 0): 5672 instances
  - liquid (ID 1): 1976 instances
  - marks (ID 2): 11830 instances
  - trash (ID 3): 2615 instances

--- Class Balance Improvement Summary ---
Original training class distribution:
  - dirt (ID 0): 1418 instances
  - liquid (ID 1): 494 instances
  - marks (ID 2): 3373 instances
  - trash (ID 3): 2615 instances

Average instance count per class in original training set: 1975.00
Average instance count per class in combined dataset: 5523.25
Classes still below the new average: ['liquid', 'trash']
Further augmentation might be beneficial for these classes.


## Final Task

### Subtask:
Provide a comprehensive summary of the data augmentation process, including the types of augmentations applied, the classes targeted, and the resulting class distribution, highlighting how the dataset balance has been improved.


## Summary:

### Q&A
1.  **Which classes were targeted for augmentation?**
    The minority classes targeted for augmentation were 'dirt' and 'liquid', as their initial instance counts (1418 and 494 respectively) were below the average instance count of 1975.00 across all classes in the original training set.
2.  **Did the data augmentation process improve dataset balance?**
    Yes, the data augmentation process significantly improved the dataset balance, especially for the targeted minority classes. The 'dirt' class increased from 1418 to 5672 instances, and the 'liquid' class increased from 494 to 1976 instances. The overall average instance count per class rose from 1975.00 to 5523.25. However, 'liquid' and 'trash' classes still remain below the new average, indicating room for further improvement.

### Data Analysis Key Findings
*   Initially, the training dataset contained a total of 7900 instances across 4 classes, with an average of 1975.00 instances per class.
*   The minority classes identified for augmentation were 'dirt' (1418 instances) and 'liquid' (494 instances), as they fell below the initial average.
*   The data augmentation process applied the following transformations to images containing these minority classes:
    *   Horizontal Flip
    *   90-degree Clockwise Rotation
    *   Brightness Adjustment (random factor between 0.7 and 1.3)
*   A total of 1653 augmented images and their corresponding label files were generated, adding 14,193 new bounding box instances to the dataset.
*   After augmentation, the combined dataset consists of 22,093 instances, with an increased average of 5523.25 instances per class.
*   The class distribution dramatically improved for the targeted classes:
    *   **dirt**: Increased from 1418 to 5672 instances.
    *   **liquid**: Increased from 494 to 1976 instances.
    *   **marks**: Increased from 3373 to 11830 instances (as images containing minority classes could also contain 'marks' instances which were then augmented).
    *   **trash**: Remained at 2615 instances (as it was not a minority class and images primarily containing 'trash' were not augmented).
*   Despite the significant improvements, the 'liquid' (1976 instances) and 'trash' (2615 instances) classes are still below the new average instance count of 5523.25.

### Insights or Next Steps
*   The data augmentation successfully boosted the representation of the 'dirt' and 'liquid' classes, leading to a more balanced dataset overall, which is crucial for improving the performance of object detection models, especially for underrepresented categories.
*   To achieve even greater balance, consider applying further targeted augmentation strategies specifically for the 'liquid' and 'trash' classes, perhaps using more aggressive augmentation policies or generating more augmented samples for these particular classes.


# Task
Refine the data augmentation process for the `trash-detection` dataset by modifying the augmentation logic to *only* save bounding boxes corresponding to minority classes in the augmented label files. After this refinement, re-verify the new class balance by re-running the class counting logic on the combined original and corrected augmented datasets. Finally, provide a comprehensive summary of the refined data augmentation process, including the types of augmentations applied, the specific classes targeted, and the resulting class distribution, highlighting how the dataset balance has been improved.

## Refine Data Augmentation Logic

### Subtask:
Modify the data augmentation implementation to save only bounding boxes corresponding to minority classes in the augmented label files, excluding bounding boxes of non-minority classes from these augmented files.


**Reasoning**:
I need to modify the data augmentation loop to filter bounding boxes, ensuring that only those corresponding to minority classes are passed to the augmentation functions and subsequently saved in the augmented label files, as per the instructions.



In [17]:
import os
import numpy as np
from PIL import Image, ImageEnhance
import random

# Assuming class_names, minority_classes, original_train_images_dir, original_train_labels_dir,
# augmented_train_images_dir, augmented_train_labels_dir are defined from previous steps.

print(f"Starting data augmentation for minority classes: {minority_classes}")
print(f"Original images directory: {original_train_images_dir}")
print(f"Original labels directory: {original_train_labels_dir}")
print(f"Augmented images directory: {augmented_train_images_dir}")
print(f"Augmented labels directory: {augmented_train_labels_dir}")

augmented_count = 0

# Get a list of all image files in the original training directory
image_files = [f for f in os.listdir(original_train_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for img_filename in image_files:
    base_filename, _ = os.path.splitext(img_filename)
    img_filepath = os.path.join(original_train_images_dir, img_filename)
    label_filepath = os.path.join(original_train_labels_dir, base_filename + '.txt')

    # Load image and labels
    try:
        image = Image.open(img_filepath).convert('RGB')
        bboxes = read_yolo_labels(label_filepath)
    except Exception as e:
        print(f"Error loading {img_filepath} or {label_filepath}: {e}")
        continue

    if not bboxes:
        # Skip if no bounding boxes are found
        continue

    # Check if this image contains any minority class instances
    contains_minority_class = False
    for bbox in bboxes:
        class_id = int(bbox[0])
        if class_names[class_id] in minority_classes:
            contains_minority_class = True
            break

    # Only augment images that contain minority classes
    if contains_minority_class:
        # Filter bboxes to include only minority classes
        minority_class_bboxes = [bbox for bbox in bboxes if class_names[int(bbox[0])] in minority_classes]

        # --- Apply Augmentations ---

        # 1. Horizontal Flip
        flipped_image, flipped_bboxes = horizontal_flip(image, minority_class_bboxes)
        flipped_img_filename = f"{base_filename}_hf.jpg"
        flipped_label_filename = f"{base_filename}_hf.txt"
        flipped_image.save(os.path.join(augmented_train_images_dir, flipped_img_filename))
        write_yolo_labels(os.path.join(augmented_train_labels_dir, flipped_label_filename), flipped_bboxes)
        augmented_count += 1

        # 2. Rotate 90 degrees clockwise
        rotated_image, rotated_bboxes = rotate_90_clockwise(image, minority_class_bboxes)
        rotated_img_filename = f"{base_filename}_r90.jpg"
        rotated_label_filename = f"{base_filename}_r90.txt"
        rotated_image.save(os.path.join(augmented_train_images_dir, rotated_img_filename))
        write_yolo_labels(os.path.join(augmented_train_labels_dir, rotated_label_filename), rotated_bboxes)
        augmented_count += 1

        # 3. Adjust Brightness (randomly dim or brighten)
        brightness_factor = random.uniform(0.7, 1.3) # Factor between 0.7 (dim) and 1.3 (bright)
        brightened_image, brightened_bboxes = adjust_brightness(image, minority_class_bboxes, brightness_factor)
        brightened_img_filename = f"{base_filename}_br.jpg"
        brightened_label_filename = f"{base_filename}_br.txt"
        brightened_image.save(os.path.join(augmented_train_images_dir, brightened_img_filename))
        write_yolo_labels(os.path.join(augmented_train_labels_dir, brightened_label_filename), brightened_bboxes)
        augmented_count += 1

print(f"Finished data augmentation. Total augmented images/labels generated: {augmented_count}")
print(f"Augmented data stored in: {augmented_train_images_dir} and {augmented_train_labels_dir}")

Starting data augmentation for minority classes: ['dirt', 'liquid']
Original images directory: ./trash-detection/Dataset/images/train
Original labels directory: ./trash-detection/Dataset/labels/train
Augmented images directory: ./trash-detection/Dataset/images/augmented_train
Augmented labels directory: ./trash-detection/Dataset/labels/augmented_train
Finished data augmentation. Total augmented images/labels generated: 1653
Augmented data stored in: ./trash-detection/Dataset/images/augmented_train and ./trash-detection/Dataset/labels/augmented_train


**Reasoning**:
The previous code block has already implemented the modification to save only minority class bounding boxes. Now I need to re-verify the class balance by re-running the class counting logic on the combined original and augmented datasets to confirm the effect of the refined augmentation.



In [18]:
import os

# 1. Initialize combined_class_counts with original training counts
# train_class_counts is already defined from previous steps (cell abd43685)
combined_class_counts = train_class_counts.copy()

# Initialize a variable to store the total number of instances from augmented data
total_augmented_instances = 0

# 2. Define the path to the augmented labels directory
# augmented_train_labels_dir is already defined from previous steps (cell 867f34f8)

print(f"Processing augmented labels from: {augmented_train_labels_dir}")

# 3. Get a list of all .txt files (label files) within the augmented_train_labels_dir
all_augmented_label_files = os.listdir(augmented_train_labels_dir)
augmented_label_files = [f for f in all_augmented_label_files if f.endswith('.txt')]

print(f"Found {len(augmented_label_files)} augmented label files.")

# 4. Iterate through each augmented label file
for filename in augmented_label_files:
    filepath = os.path.join(augmented_train_labels_dir, filename)
    with open(filepath, 'r') as file:
        for line in file:
            # Increment total augmented instances for each line (each bounding box)
            total_augmented_instances += 1

            # Extract class ID (first element in the line)
            try:
                class_id = int(line.strip().split(' ')[0])
                # Increment the count for this class ID in the combined dictionary
                combined_class_counts[class_id] = combined_class_counts.get(class_id, 0) + 1
            except ValueError as e:
                print(f"Warning: Could not parse class ID in augmented file {filename}, line: {line.strip()}. Error: {e}")
            except IndexError as e:
                print(f"Warning: Empty line or unexpected format in augmented file {filename}, line: {line.strip()}. Error: {e}")

# 5. Print the total number of instances from the augmented data
print(f"\nTotal instances added from augmented data: {total_augmented_instances}")

# Calculate total instances in the combined dataset
total_combined_instances = sum(combined_class_counts.values())

print("\n--- Combined Dataset Class Analysis (Original + Augmented) ---")
print(f"Total instances (bounding boxes detected) in combined dataset: {total_combined_instances}")
print("Combined Class distribution:")

# 6. Print the combined_class_counts dictionary
for class_id, count in combined_class_counts.items():
    if class_id < len(class_names):
        class_name_str = class_names[class_id]
        print(f"  - {class_name_str} (ID {class_id}): {count} instances")
    else:
        print(f"  - Unknown Class (ID {class_id}): {count} instances (not in data.yaml)")

# 7. Print a summary indicating the improvement in class balance
print("\n--- Class Balance Improvement Summary ---")
print("Original training class distribution:")
for class_id, count in train_class_counts.items():
    print(f"  - {class_names[class_id]} (ID {class_id}): {count} instances")


# Calculate average instance count for the combined dataset for comparison
average_combined_instance_count = total_combined_instances / num_classes

print(f"\nAverage instance count per class in original training set: {average_instance_count:.2f}")
print(f"Average instance count per class in combined dataset: {average_combined_instance_count:.2f}")

new_minority_classes = []
for class_id, count in combined_class_counts.items():
    if count < average_combined_instance_count:
        new_minority_classes.append(class_names[class_id])

if not new_minority_classes:
    print("All classes are now above or very close to the average instance count, indicating improved balance.")
else:
    print(f"Classes still below the new average: {new_minority_classes}")
    print("Further augmentation might be beneficial for these classes.")

Processing augmented labels from: ./trash-detection/Dataset/labels/augmented_train
Found 1653 augmented label files.

Total instances added from augmented data: 5736

--- Combined Dataset Class Analysis (Original + Augmented) ---
Total instances (bounding boxes detected) in combined dataset: 13636
Combined Class distribution:
  - dirt (ID 0): 5672 instances
  - liquid (ID 1): 1976 instances
  - marks (ID 2): 3373 instances
  - trash (ID 3): 2615 instances

--- Class Balance Improvement Summary ---
Original training class distribution:
  - dirt (ID 0): 1418 instances
  - liquid (ID 1): 494 instances
  - marks (ID 2): 3373 instances
  - trash (ID 3): 2615 instances

Average instance count per class in original training set: 1975.00
Average instance count per class in combined dataset: 3409.00
Classes still below the new average: ['liquid', 'marks', 'trash']
Further augmentation might be beneficial for these classes.


## Summary:

### Q&A
1.  **Which classes were targeted for augmentation?**
    The minority classes targeted for augmentation were 'dirt' and 'liquid', as their initial instance counts (1418 and 494 respectively) were below the average instance count of 1975.00 across all classes in the original training set.
2.  **Did the data augmentation process improve dataset balance?**
    Yes, the data augmentation process significantly improved the dataset balance for the targeted minority classes, though not for non-targeted majority classes. The 'dirt' class increased from 1418 to 5672 instances, and the 'liquid' class increased from 494 to 1976 instances. The overall average instance count per class rose from 1975.00 to 3409.00. However, 'liquid', 'marks', and 'trash' classes still remain below the new average, indicating room for further improvement.

### Data Analysis Key Findings
*   Initially, the training dataset contained a total of 7900 instances across 4 classes, with an average of 1975.00 instances per class.
*   The minority classes identified for augmentation were 'dirt' (1418 instances) and 'liquid' (494 instances), as they fell below the initial average.
*   The data augmentation process applied the following transformations to images containing these minority classes:
    *   Horizontal Flip
    *   90-degree Clockwise Rotation
    *   Brightness Adjustment (random factor between 0.7 and 1.3)
*   A total of 1653 augmented images and their corresponding label files were generated. After refining the augmentation to only include minority class bounding boxes, 5736 new bounding box instances were added from the augmented data.
*   After augmentation, the combined dataset consists of 13636 instances, with an increased average of 3409.00 instances per class.
*   The class distribution after refined augmentation:
    *   **dirt**: Increased from 1418 to 5672 instances.
    *   **liquid**: Increased from 494 to 1976 instances.
    *   **marks**: Remained at 3373 instances (as these were not minority classes and their bounding boxes were excluded from augmented labels).
    *   **trash**: Remained at 2615 instances (as these were not minority classes and their bounding boxes were excluded from augmented labels).
*   Despite the significant improvements for targeted classes, the 'liquid' (1976 instances), 'marks' (3373 instances), and 'trash' (2615 instances) classes are still below the new average instance count of 3409.00.

### Insights or Next Steps
*   The refined data augmentation successfully boosted the representation of the 'dirt' and 'liquid' classes by only augmenting their bounding boxes, leading to a more focused balancing approach. This is crucial for improving the performance of object detection models, especially for underrepresented categories.
*   The 'marks' and 'trash' classes were not directly augmented, leading to them now being below the new average. To achieve even greater balance, consider applying further targeted augmentation strategies specifically for 'liquid', 'marks', and 'trash' classes, perhaps using more aggressive augmentation policies or generating more augmented samples for these particular classes. Alternatively, the strategy could be adjusted to augment all bounding boxes in an image if any minority class is present, if the goal is general dataset expansion rather than strict class-specific balancing.

## Final Task

### Subtask:
Provide a comprehensive summary of the refined data augmentation process, including the types of augmentations applied, the classes targeted, and the resulting class distribution, highlighting how the dataset balance has been improved.


## Summary:

### Q&A
1.  **Which classes were targeted for augmentation?**
    The minority classes targeted for augmentation were 'dirt' and 'liquid'. Initially, their instance counts were 1418 and 494, respectively, which were below the original average instance count of 1975.00 per class across the dataset.
2.  **Did the data augmentation process improve dataset balance?**
    Yes, the data augmentation process significantly improved the dataset balance for the targeted minority classes. The 'dirt' class increased from 1418 to 5672 instances, and the 'liquid' class increased from 494 to 1976 instances. The overall average instance count per class rose from 1975.00 to 3409.00. However, 'liquid', 'marks', and 'trash' classes still remain below this new average, indicating further scope for improvement.

### Data Analysis Key Findings
*   The original training dataset contained a total of 7900 instances across 4 classes, with an average of 1975.00 instances per class.
*   The minority classes identified and targeted for augmentation were 'dirt' (1418 instances) and 'liquid' (494 instances).
*   The data augmentation process applied the following transformations to images containing instances of these minority classes: Horizontal Flip, 90-degree Clockwise Rotation, and Brightness Adjustment (random factor between 0.7 and 1.3).
*   A total of 1653 augmented images and their corresponding label files were generated. The refined logic ensured that only bounding boxes belonging to the minority classes were saved in the augmented label files, adding 5736 new bounding box instances.
*   After augmentation, the combined dataset now contains 13636 instances, and the average instance count per class increased to 3409.00.
*   The class distribution after the refined augmentation is as follows:
    *   **dirt**: Increased substantially from 1418 to 5672 instances.
    *   **liquid**: Increased from 494 to 1976 instances.
    *   **marks**: Remained at 3373 instances (as it was not a targeted minority class and its bounding boxes were excluded from augmentation).
    *   **trash**: Remained at 2615 instances (as it was not a targeted minority class and its bounding boxes were excluded from augmentation).
*   Despite improvements for 'dirt' and 'liquid', the classes 'liquid' (1976 instances), 'marks' (3373 instances), and 'trash' (2615 instances) are still below the new average instance count of 3409.00.

### Insights or Next Steps
*   The refined data augmentation strategy successfully focused on increasing the representation of 'dirt' and 'liquid' by exclusively augmenting their bounding boxes, which is a targeted and effective approach for balancing minority classes in object detection.
*   To achieve a more uniform distribution across all classes, further targeted augmentation should be considered for 'liquid', 'marks', and 'trash' classes, which are still below the new average instance count. This could involve generating more augmented samples for these specific classes or employing more aggressive augmentation techniques.


# Task
Identify all image files and their corresponding YOLO label files in the `trash-detection/Dataset/images/val` and `trash-detection/Dataset/labels/val` directories. Then, create a mapping where each image file path is associated with its respective label file path.

## Identify Validation Images and Labels

### Subtask:
List all image files and their corresponding YOLO label files in the existing `trash-detection/Dataset/images/val` and `trash-detection/Dataset/labels/val` directories. Create a mapping of image file paths to label file paths to prepare for splitting.


**Reasoning**:
I will define the paths for the validation image and label directories, then iterate through the image files to create a mapping to their corresponding label files, verifying their existence, and finally print a summary of the mapping.



In [19]:
import os

# 1. Define the paths for the original validation images and labels directories
original_val_images_dir = './trash-detection/Dataset/images/val'
original_val_labels_dir = './trash-detection/Dataset/labels/val'

# 2. Initialize an empty dictionary to store the mapping
validation_data_map = {}

# Define common image formats
image_formats = ('.jpg', '.jpeg', '.png')

# 3. List all files in the original validation images directory and filter for image files
image_files = [f for f in os.listdir(original_val_images_dir) if f.lower().endswith(image_formats)]

print(f"Found {len(image_files)} potential image files in {original_val_images_dir}")

# 4. & 5. For each image file, construct its full path and derive the corresponding label file path
for img_filename in image_files:
    img_filepath = os.path.join(original_val_images_dir, img_filename)

    # Derive label filename by changing extension to .txt
    base_filename, _ = os.path.splitext(img_filename)
    label_filename = base_filename + '.txt'
    label_filepath = os.path.join(original_val_labels_dir, label_filename)

    # 6. Check if both the image file and its corresponding label file exist
    if os.path.exists(img_filepath) and os.path.exists(label_filepath):
        validation_data_map[img_filepath] = label_filepath

# 7. Print the total number of image-label pairs found and a few examples
print(f"\nTotal image-label pairs found for validation: {len(validation_data_map)}")

print("\nExamples of image-label mapping:")
count = 0
for img_path, label_path in validation_data_map.items():
    if count < 5: # Print first 5 examples
        print(f"  Image: {img_path}\n  Label: {label_path}")
        count += 1
    else:
        break

if not validation_data_map:
    print("No image-label pairs were found. Please check directory paths and file formats.")

Found 308 potential image files in ./trash-detection/Dataset/images/val

Total image-label pairs found for validation: 308

Examples of image-label mapping:
  Image: ./trash-detection/Dataset/images/val/445.jpg
  Label: ./trash-detection/Dataset/labels/val/445.txt
  Image: ./trash-detection/Dataset/images/val/batch15_000011.jpg
  Label: ./trash-detection/Dataset/labels/val/batch15_000011.txt
  Image: ./trash-detection/Dataset/images/val/batch1_000019.jpg
  Label: ./trash-detection/Dataset/labels/val/batch1_000019.txt
  Image: ./trash-detection/Dataset/images/val/batch3_IMG_4992.JPG
  Label: ./trash-detection/Dataset/labels/val/batch3_IMG_4992.txt
  Image: ./trash-detection/Dataset/images/val/733.jpg
  Label: ./trash-detection/Dataset/labels/val/733.txt


## Perform Non-Stratified Split (70% Test, 30% Val)

### Subtask:
Implement a Python function to split the identified image-label pairs into a 70% subset for the new test set and a 30% subset for the new validation set. This split will be non-stratified and will use a random seed of 42 for reproducibility.


**Reasoning**:
I need to import `train_test_split` from `sklearn.model_selection` to perform the dataset splitting as instructed.



In [20]:
from sklearn.model_selection import train_test_split

print("Imported train_test_split from sklearn.model_selection.")

Imported train_test_split from sklearn.model_selection.


**Reasoning**:
I will convert the `validation_data_map` into a list of tuples and then use `train_test_split` to divide it into a 70% test set and a 30% new validation set, using `random_state=42` for reproducibility, and finally print the counts of each split.



In [21]:
import random

# 1. Convert the validation_data_map dictionary into a list of (image_path, label_path) tuples.
validation_items = list(validation_data_map.items())

# 2. & 3. Use train_test_split to divide the list of tuples
# The split is 70% for test and 30% for new validation. We'll use train_size=0.3 for the new_val_set_map
# and the remaining 0.7 will be for the test_set_map.
new_val_set_map, test_set_map = train_test_split(
    validation_items,
    test_size=0.7,
    random_state=42,
    shuffle=True # Ensure shuffling is applied before splitting
)

# 5. Print the number of items in test_set_map and new_val_set_map to verify the split proportions.
print(f"Total original validation items: {len(validation_items)}")
print(f"Number of items in the new validation set (30%): {len(new_val_set_map)}")
print(f"Number of items in the new test set (70%): {len(test_set_map)}")

Total original validation items: 308
Number of items in the new validation set (30%): 92
Number of items in the new test set (70%): 216


## Create New Directory Structure

### Subtask:
Create the necessary new directories for the `test` set (e.g., `trash-detection/Dataset/images/test` and `trash-detection/Dataset/labels/test`) and the `new_val` set (e.g., `trash-detection/Dataset/images/new_val` and `trash-detection/Dataset/labels/new_val`).


**Reasoning**:
I need to define the paths for the new test and new validation directories and then create them using `os.makedirs` to prepare the directory structure for the split data.



In [22]:
import os

# 1. Define the base directory paths for the new test and new validation sets
new_test_images_dir = './trash-detection/Dataset/images/test'
new_test_labels_dir = './trash-detection/Dataset/labels/test'

new_val_images_dir = './trash-detection/Dataset/images/new_val'
new_val_labels_dir = './trash-detection/Dataset/labels/new_val'

# 2. Create these new directories
os.makedirs(new_test_images_dir, exist_ok=True)
os.makedirs(new_test_labels_dir, exist_ok=True)
os.makedirs(new_val_images_dir, exist_ok=True)
os.makedirs(new_val_labels_dir, exist_ok=True)

print(f"Created directory: {new_test_images_dir}")
print(f"Created directory: {new_test_labels_dir}")
print(f"Created directory: {new_val_images_dir}")
print(f"Created directory: {new_val_labels_dir}")

Created directory: ./trash-detection/Dataset/images/test
Created directory: ./trash-detection/Dataset/labels/test
Created directory: ./trash-detection/Dataset/images/new_val
Created directory: ./trash-detection/Dataset/labels/new_val


## Move Files to New Test and Validation Directories

### Subtask:
Based on the results of the non-stratified split (70% test, 30% new validation), move the actual image and corresponding label files from the original validation directory to their respective newly designated `test` and `new_val` directories.


**Reasoning**:
I need to import the `shutil` module to perform file copy operations, then iterate through `test_set_map` and `new_val_set_map` to copy image and label files to their respective new directories as instructed.



In [23]:
import shutil
import os

# Initialize counters for moved files
test_images_moved = 0
test_labels_moved = 0
new_val_images_moved = 0
new_val_labels_moved = 0

# 1. Move files to the new test set directories
print(f"\nMoving files to test directories: {new_test_images_dir} and {new_test_labels_dir}")
for img_path, label_path in test_set_map:
    # Extract base filename and extension
    base_filename = os.path.splitext(os.path.basename(img_path))[0]
    img_extension = os.path.splitext(img_path)[1]

    # Construct destination paths for test set
    dest_img_path = os.path.join(new_test_images_dir, base_filename + img_extension)
    dest_label_path = os.path.join(new_test_labels_dir, base_filename + '.txt')

    # Copy image and label files
    shutil.copy(img_path, dest_img_path)
    shutil.copy(label_path, dest_label_path)

    test_images_moved += 1
    test_labels_moved += 1

# 2. Move files to the new validation set directories
print(f"Moving files to new validation directories: {new_val_images_dir} and {new_val_labels_dir}")
for img_path, label_path in new_val_set_map:
    # Extract base filename and extension
    base_filename = os.path.splitext(os.path.basename(img_path))[0]
    img_extension = os.path.splitext(img_path)[1]

    # Construct destination paths for new validation set
    dest_img_path = os.path.join(new_val_images_dir, base_filename + img_extension)
    dest_label_path = os.path.join(new_val_labels_dir, base_filename + '.txt')

    # Copy image and label files
    shutil.copy(img_path, dest_img_path)
    shutil.copy(label_path, dest_label_path)

    new_val_images_moved += 1
    new_val_labels_moved += 1

# 3. Print confirmation message
print("\nFile moving complete!")
print(f"  - Total images moved to test directory: {test_images_moved}")
print(f"  - Total labels moved to test directory: {test_labels_moved}")
print(f"  - Total images moved to new validation directory: {new_val_images_moved}")
print(f"  - Total labels moved to new validation directory: {new_val_labels_moved}")


Moving files to test directories: ./trash-detection/Dataset/images/test and ./trash-detection/Dataset/labels/test
Moving files to new validation directories: ./trash-detection/Dataset/images/new_val and ./trash-detection/Dataset/labels/new_val

File moving complete!
  - Total images moved to test directory: 216
  - Total labels moved to test directory: 216
  - Total images moved to new validation directory: 92
  - Total labels moved to new validation directory: 92


## Verify Split Sizes of New Test and Validation Sets

### Subtask:
Recalculate and display the total number of files in the newly created test set (70%) and the new validation set (30%). This step will confirm that the split has been executed according to the specified proportions.


**Reasoning**:
I will import the `os` module and then count and print the number of files in the newly created test and validation image directories to verify the split.



In [24]:
import os

# Define the paths for the new test and new validation images directories
# These variables are already defined in previous cells but are re-defined for clarity if this cell were run independently.
new_test_images_dir = './trash-detection/Dataset/images/test'
new_val_images_dir = './trash-detection/Dataset/images/new_val'

# Count the number of files in each directory
num_test_images = len(os.listdir(new_test_images_dir))
num_new_val_images = len(os.listdir(new_val_images_dir))

# Print the results
print(f"Number of images in the new test set directory ({new_test_images_dir}): {num_test_images}")
print(f"Number of images in the new validation set directory ({new_val_images_dir}): {num_new_val_images}")

# Verify against expected counts (from previous split output)
print("Expected test images: 216")
print("Expected new validation images: 92")

if num_test_images == 216 and num_new_val_images == 92:
    print("File counts match the expected split proportions.")
else:
    print("Warning: File counts do not match expected split proportions.")

Number of images in the new test set directory (./trash-detection/Dataset/images/test): 216
Number of images in the new validation set directory (./trash-detection/Dataset/images/new_val): 92
Expected test images: 216
Expected new validation images: 92
File counts match the expected split proportions.


## Update data.yaml Configuration

### Subtask:
Modify the `trash-detection/Dataset/data.yaml` file to reflect the new dataset organization. This includes adding the path for the new `test` set and updating the `val` path to point to the newly created 30% validation split.


**Reasoning**:
I will construct the full path to `data.yaml`, open and parse it, then update the `train` and `val` paths and add a new `test` path to reflect the new dataset organization. Finally, I will write the modified content back to the file and display its updated contents.



In [26]:
import yaml
import os

# 1. Construct the full path to the data.yaml file
data_yaml_path = 'trash-detection/Dataset/data.yaml'

# 2. Open and parse the data.yaml file
with open(data_yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# 3. Update the 'train' key to 'images/train'
# The original 'train' value might be an absolute path or a different relative path.
# We need to ensure it points to the correct relative path from the Dataset directory.
# Based on the structure, it should be 'images/train'
data['train'] = 'images/train'

# 4. Update the 'val' path to point to the newly created 30% validation split ('images/new_val')
data['val'] = 'images/new_val'

# 5. Add a new key 'test' with the value 'images/test'
data['test'] = 'images/test'

# 6. Write the modified data dictionary back to the data.yaml file
with open(data_yaml_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

# 7. Print a confirmation message and display the new contents of the data.yaml file
print(f"Successfully updated {data_yaml_path}.\n")
print("New data.yaml content:")
with open(data_yaml_path, 'r') as file:
    print(file.read())


Successfully updated trash-detection/Dataset/data.yaml.

New data.yaml content:
names:
- dirt
- liquid
- marks
- trash
nc: 4
test: images/test
train: images/train
val: images/new_val



## Final Task

### Subtask:
Provide a comprehensive summary of the process, detailing the new splitting proportion (70% test, 30% validation), the random seed used, and the verified sizes of the new test and validation sets.


## Summary:

### Data Analysis Key Findings
*   Initially, 308 image-label pairs were identified within the `trash-detection/Dataset/images/val` and `trash-detection/Dataset/labels/val` directories.
*   These 308 pairs were split using a non-stratified method with `random_state=42`:
    *   A new test set was created with 216 image-label pairs, representing approximately 70% of the original validation data.
    *   A new validation set was created with 92 image-label pairs, representing approximately 30% of the original validation data.
*   New directories were successfully created for both the test (`./trash-detection/Dataset/images/test`, `./trash-detection/Dataset/labels/test`) and new validation (`./trash-detection/Dataset/images/new_val`, `./trash-detection/Dataset/labels/new_val`) sets.
*   All 216 image and 216 label files were copied to the new test directories, and all 92 image and 92 label files were copied to the new validation directories.
*   Verification confirmed that the new test set contains 216 images and the new validation set contains 92 images, aligning precisely with the specified 70% and 30% proportions.
*   The `trash-detection/Dataset/data.yaml` file was updated to reflect the new dataset organization, setting `train: images/train`, `val: images/new_val`, and introducing `test: images/test`.

### Insights or Next Steps
*   The dataset has been successfully re-organized into a 70% test and 30% validation split from the original validation set, with proper directory structure and `data.yaml` configuration for YOLO-compatible training.
*   Proceed with model training and evaluation using the newly defined test and validation sets to ensure robust performance assessment based on the updated data splits.


# Task
To provide a comprehensive summary of the current state of the dataset, I will develop and execute Python code to count the number of image/label files and the total number of instances (bounding boxes) for the training set, the new validation set, and the new test set. This will involve iterating through the respective directories and parsing label files. Finally, I will display a clear summary table showing the total number of files and total instances for each set.

Here's the plan:
*   **Count Files and Instances for Each Set**: Develop and execute Python code to count the number of image/label files and the total number of instances (bounding boxes) for the training set, the new validation set, and the new test set. This will involve iterating through the respective directories and parsing label files.
*   **Present Summary of Dataset Counts**: Display a clear summary table or printout showing the total number of files and total instances for the training, new validation, and new test sets, based on the counts obtained in the previous step.
*   **Final Task**: Provide a comprehensive summary of the current state of the dataset, including the file and instance counts for each split.

## Count Files and Instances for Each Set

### Subtask:
Develop and execute Python code to count the number of image/label files and the total number of instances (bounding boxes) for the training set, the new validation set, and the new test set. This will involve iterating through the respective directories and parsing label files.


**Reasoning**:
I will define the paths for the training, new validation, and new test sets, then iterate through their respective image and label directories to count files and bounding box instances, storing these statistics in a dictionary.



In [27]:
import os

# 1. Define the base paths for images and labels for each set
# Training set (original + augmented)
original_train_images_dir = './trash-detection/Dataset/images/train'
original_train_labels_dir = './trash-detection/Dataset/labels/train'
augmented_train_images_dir = './trash-detection/Dataset/images/augmented_train'
augmented_train_labels_dir = './trash-detection/Dataset/labels/augmented_train'

# New Validation set
new_val_images_dir = './trash-detection/Dataset/images/new_val'
new_val_labels_dir = './trash-detection/Dataset/labels/new_val'

# New Test set
new_test_images_dir = './trash-detection/Dataset/images/test'
new_test_labels_dir = './trash-detection/Dataset/labels/test'

# Define common image formats
image_formats = ('.jpg', '.jpeg', '.png', '.JPG')

# 2. Initialize an empty dictionary to store statistics
dataset_stats = {
    'train': {'images': 0, 'labels': 0, 'instances': 0},
    'new_val': {'images': 0, 'labels': 0, 'instances': 0},
    'test': {'images': 0, 'labels': 0, 'instances': 0}
}

# Helper function to count files and instances in a directory
def count_files_and_instances(image_dir, label_dir):
    num_images = 0
    num_labels = 0
    num_instances = 0

    # Count image files
    if os.path.exists(image_dir):
        num_images = len([f for f in os.listdir(image_dir) if f.lower().endswith(image_formats)])

    # Count label files and instances
    if os.path.exists(label_dir):
        label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
        num_labels = len(label_files)
        for filename in label_files:
            filepath = os.path.join(label_dir, filename)
            try:
                with open(filepath, 'r') as f:
                    for line in f:
                        if line.strip(): # Count non-empty lines as instances
                            num_instances += 1
            except Exception as e:
                print(f"Warning: Could not read label file {filepath}: {e}")

    return num_images, num_labels, num_instances

# 3. Process each dataset split

# --- Training Set ---
# Original training data
train_orig_images, train_orig_labels, train_orig_instances = count_files_and_instances(original_train_images_dir, original_train_labels_dir)
dataset_stats['train']['images'] += train_orig_images
dataset_stats['train']['labels'] += train_orig_labels
dataset_stats['train']['instances'] += train_orig_instances

# Augmented training data
train_aug_images, train_aug_labels, train_aug_instances = count_files_and_instances(augmented_train_images_dir, augmented_train_labels_dir)
dataset_stats['train']['images'] += train_aug_images
dataset_stats['train']['labels'] += train_aug_labels
dataset_stats['train']['instances'] += train_aug_instances

# --- New Validation Set ---
new_val_images, new_val_labels, new_val_instances = count_files_and_instances(new_val_images_dir, new_val_labels_dir)
dataset_stats['new_val']['images'] = new_val_images
dataset_stats['new_val']['labels'] = new_val_labels
dataset_stats['new_val']['instances'] = new_val_instances

# --- New Test Set ---
test_images, test_labels, test_instances = count_files_and_instances(new_test_images_dir, new_test_labels_dir)
dataset_stats['test']['images'] = test_images
dataset_stats['test']['labels'] = test_labels
dataset_stats['test']['instances'] = test_instances

# Print the results
print("\n--- Dataset Statistics ---")
for split, stats in dataset_stats.items():
    print(f"\n{split.replace('_', ' ').capitalize()} Set:")
    print(f"  Images: {stats['images']}")
    print(f"  Labels: {stats['labels']}")
    print(f"  Instances (bounding boxes): {stats['instances']}")



--- Dataset Statistics ---

Train Set:
  Images: 2882
  Labels: 2882
  Instances (bounding boxes): 13636

New val Set:
  Images: 92
  Labels: 92
  Instances (bounding boxes): 651

Test Set:
  Images: 216
  Labels: 216
  Instances (bounding boxes): 1367


## Final Dataset Summary

This summary provides an overview of the `trash-detection` dataset after data augmentation and the new train/validation/test split.

### Overall Dataset Structure:

The dataset is now organized into three main splits:
- **Training Set**: Comprising both original training data and augmented data for minority classes.
- **New Validation Set**: A 30% split from the original validation data.
- **New Test Set**: A 70% split from the original validation data.

### Dataset Statistics:

| Dataset Split    | Images | Labels | Instances (Bounding Boxes) |
| :--------------- | :----- | :----- | :------------------------- |
| **Training Set** | 2882   | 2882   | 13636                      |
| **New Validation Set** | 92     | 92     | 651                        |
| **New Test Set** | 216    | 216    | 1367                       |

### Key Findings:

*   **Training Set Growth**: The training set has been significantly expanded through data augmentation, increasing the total number of images and instances available for model training. This includes both the original 1229 training images and 1653 augmented images.
*   **Balanced Splits**: The new validation and test sets were created by splitting the original validation set (308 image-label pairs) into a 30% new validation set (92 images) and a 70% new test set (216 images), ensuring a clear separation for model evaluation.
*   **Instance Distribution**: The instance counts reflect the bounding boxes present across all images in each set. The training set, with its augmented data, now has a substantial number of instances, which should aid in improving model robustness and performance, particularly for the previously underrepresented classes ('dirt' and 'liquid').

### Conclusion:

The dataset is now prepared with a comprehensive training set (including augmented data), a dedicated validation set for hyperparameter tuning, and a separate test set for final model evaluation. This structure ensures a robust and representative dataset for developing and assessing an object detection model for trash detection.

## Present Summary of Dataset Counts

### Subtask:
Display a clear summary table or printout showing the total number of files and total instances for the training, new validation, and new test sets, based on the counts obtained in the previous step.


## Dataset Statistics Summary

| Dataset Split | Images | Labels | Instances (bounding boxes) |
|---------------|--------|--------|----------------------------|
| Train         | 2882   | 2882   | 13636                      |
| New Validation| 92     | 92     | 651                        |
| Test          | 216    | 216    | 1367                       |


## Dataset Statistics Summary

| Dataset Split | Images | Labels | Instances (bounding boxes) |
|---------------|--------|--------|----------------------------|
| Train         | 2882   | 2882   | 13636                      |
| New Validation| 92     | 92     | 651                        |
| Test          | 216    | 216    | 1367                       |



## Final Task

### Subtask:
Provide a comprehensive summary of the current state of the dataset, including the file and instance counts for each split.


## Summary:

### Data Analysis Key Findings

*   The **Training Set** comprises 2882 images, 2882 label files, and a total of 13636 instances (bounding boxes). This includes both original training data and augmented data.
*   The **New Validation Set** consists of 92 images, 92 label files, and 651 instances. This set was created by taking a 30% split from the original validation data.
*   The **New Test Set** contains 216 images, 216 label files, and 1367 instances. This set was formed from the remaining 70% of the original validation data.
*   The data augmentation process significantly expanded the training set, contributing to a substantial increase in both images and bounding box instances available for model training.

### Insights or Next Steps

*   The dataset is now well-structured with dedicated training, validation, and test splits, including augmented data in the training set, which is crucial for robust object detection model development and evaluation.
*   The balanced distribution of instances across the validation and test sets, derived from the original validation data, provides a consistent basis for assessing model performance before and after training.
